## Interpolates a folder of single-frame Sacks-like calendars to a reference grid

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import tqdm
import os
import re
import copy
import datetime

In [2]:
calname = "Sacks_ZARC_fill_fill_120d"
infolder = "calendars_sacks/"+calname+"/"
outfolder = "agmerra_calendars/"+calname+"/"
reffname = "D:/data/AgMERRA/regrid/AgMERRA_2001_tavg.nc4"

infsuf = ".crop.calendar.fill.nc"
crops = ["Maize","Soybeans","Cotton"]

In [3]:
# Create output folder if it doesn't exist
if not os.path.exists(outfolder): os.makedirs(outfolder, exist_ok=True)

In [4]:
# Open reference array
# refarr = xr.open_dataarray(reffname)
refds = xr.open_dataset(reffname, decode_times = False)
refarr = refds["tavg"]
print(refarr)

<xarray.DataArray 'tavg' (time: 365, latitude: 720, longitude: 1440)>
[378432000 values with dtype=float32]
Coordinates:
  * time       (time) float32 7671.0 7672.0 7673.0 ... 8033.0 8034.0 8035.0
  * longitude  (longitude) float32 -179.875 -179.625 ... 179.625 179.875
  * latitude   (latitude) float32 89.875 89.625 89.375 ... -89.625 -89.875
Attributes:
    units:               Degrees Celsius
    description:         Average Temperature
    vMin_original_data:  -66.5
    vMax_original_data:  43.0
    vRange:              109.5


In [5]:
# Loop here
for crop in crops:
    # Open the dataset
    infname = infolder + crop + infsuf
    inarr = xr.open_dataset(infname)
    
    # Rename coordinates
#     inarr = inarr.rename({"latitude":"lat", "longitude":"lon"})

    #Actual interpolation
    outarr = inarr.interp(latitude=refarr['latitude'],longitude=refarr['longitude'],method = 'nearest')
#     outarr = inarr.interp(lat=refarr['lat'],lon=refarr['lon'],method = 'nearest')

    # Output
    outfname = outfolder + crop + infsuf
    outarr.to_netcdf(outfname)